# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities_data = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities_data.head()

,City,Humidity,Lat,Lng,Date,Cloudiness,Country,Wind Speed,Max Temp
0,tasiilaq,93,65.61,-37.64,1593123181,99,GL,9.17,39.20
1,blagodatnoye,52,45.40,42.59,1593123181,35,RU,6.71,66.20
2,rikitea,83,-23.12,-134.97,1593123182,61,PF,22.32,74.44
3,busselton,76,-33.65,115.33,1593123146,62,AU,12.15,51.01
4,khed,87,17.72,73.38,1593123182,83,IN,2.01,79.90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Get locations and Humidity
import numpy as np
city_location = cities_data[["Lat", "Lng"]].astype(np.float64)
humidity = cities_data["Humidity"]#.apply(lambda x: float(x))

fig = gmaps.figure()
# Format Layout Maps

layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

# Create Heat Layer

fig = gmaps.figure(map_type='HYBRID', layout=layout)
heat_layer = gmaps.heatmap_layer(city_location, weights=humidity, 
                                  dissipating=False, max_intensity=100,
                                  point_radius = 1)
# Add Layer
fig.add_layer(heat_layer)

# Display Figure
fig




Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
weather_criteria = cities_data.loc[(cities_data["Max Temp"] >= 70) & (cities_data["Max Temp"] <= 80) &
                                       (cities_data["Wind Speed"] <= 20) & (cities_data["Wind Speed"] <= 30)]

# Drop rows with null values.

weather_criteria.shape

(159, 9)

In [10]:
weather_criteria.dropna(axis = 0,inplace = True)

/Users/dsb/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = weather_criteria 
hotel_df.reset_index(drop = True ,inplace = True)
hotel_df

,City,Humidity,Lat,Lng,Date,Cloudiness,Country,Wind Speed,Max Temp
0,khed,87,17.72,73.38,1593123182,83,IN,2.01,79.90
1,vaini,83,-21.20,-175.20,1593123182,75,TO,6.93,75.20
2,faanui,79,-16.48,-151.75,1593123153,98,PF,11.90,78.80
3,ibra,77,22.69,58.53,1593123184,37,OM,3.80,78.94
4,avarua,60,-21.21,-159.78,1593123185,30,CK,5.82,75.20
...,...,...,...,...,...,...,...,...,...
154,bintulu,94,3.17,113.03,1593122968,20,MY,2.24,75.20
155,cape coast,84,5.11,-1.25,1593123263,97,GH,12.24,77.52
156,livezi,94,44.85,23.82,1593123265,34,RO,10.29,71.01
157,matay,49,28.42,30.78,1593123266,0,EG,17.52,76.10


In [12]:
hotel_df["Hotel Name"] = ''
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    params["location"] = f'{lat},{lng}'
    
    response = requests.get(base_url ,params=params).json()
    
    results = response["results"]
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except:
        print(f"Missing result for {city}... skip it.")
    

/Users/dsb/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Closest khed hotel is Ashirwad Lodge.


/Users/dsb/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest vaini hotel is Paradise First Hotel.
Closest faanui hotel is InterContinental Bora Bora Resort Thalasso Spa.
Closest ibra hotel is Golden Rays hotel.
Closest avarua hotel is Pacific Resort Rarotonga.
Closest ravar hotel is Hotel sperlous.
Closest okitipupa hotel is Amazon Hotel & Suite.
Closest kabanjahe hotel is OYO 1698 Hotel Juma Eluk.
Closest atuona hotel is Simplicité Marquises.
Closest guerrero negro hotel is Hotel Los Caracoles.
Closest puerto leguizamo hotel is la casona de juancho.
Closest pangkalanbuun hotel is Grand Kecubung Hotel.
Closest sao felix do xingu hotel is Hotel Serrano.
Closest ryotsu hotel is Sado Green Hotel Kiraku.
Closest mahebourg hotel is Shandrani Beachcomber Resort & Spa.
Closest thetford mines hotel is La Cache Du Domaine.
Closest damghan hotel is Damghan Tourism Hotel.
Missing result for bongor... skip it.
Closest tura hotel is Hotel Polo Orchid.
Closest arraial do cabo hotel is Orlanova Hotel.
Closest inongo hotel is Hotel Du Lac.
Closest port 

In [13]:
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df = hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})
hotel_df

,City,Humidity,Lat,Lng,Date,Cloudiness,Country,Wind Speed,Max Temp,Hotel Name
0,khed,87,17.72,73.38,1593123182,83,IN,2.01,79.90,Ashirwad Lodge
1,vaini,83,-21.20,-175.20,1593123182,75,TO,6.93,75.20,Paradise First Hotel
2,faanui,79,-16.48,-151.75,1593123153,98,PF,11.90,78.80,InterContinental Bora Bora Resort Thalasso Spa
3,ibra,77,22.69,58.53,1593123184,37,OM,3.80,78.94,Golden Rays hotel
4,avarua,60,-21.21,-159.78,1593123185,30,CK,5.82,75.20,Pacific Resort Rarotonga
...,...,...,...,...,...,...,...,...,...,...
154,bintulu,94,3.17,113.03,1593122968,20,MY,2.24,75.20,New World Suites
155,cape coast,84,5.11,-1.25,1593123263,97,GH,12.24,77.52,"Ridge Royal Hotel, Cape Coast Ghana"
156,livezi,94,44.85,23.82,1593123265,34,RO,10.29,71.01,
157,matay,49,28.42,30.78,1593123266,0,EG,17.52,76.10,


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))